### We want to get the best Prophet prediction for spare parts whose command number is between 50-100. So we use the data from 09-17 as training data and 2018 as test data. We want to find the best  prophet paramater 'espically chang_point rate'  that can mininser the overall error

In [58]:
## import and load in
import pandas as pd
import numpy as np
from fbprophet import Prophet
import os

In [61]:
# select the name whose total command is between 50-100 and save them in PARAs
df1 = pd.read_excel("encoded_pic_septembre_2018.xlsx")
df2=df1.groupby(by=['item',df1.index]).sum()
df3=df2.groupby(['item']).size()
df4 = df3.sort_values()
PARAs = df4[1450:1520].index
# 50 - 100: 1450 - 1520

Index(['Alliance', 'Ankeny', 'Darlington', 'Greensboro', 'Chelan',
       'Columbiana', 'Dayton', 'Clifton', 'Cleveland Heights', 'Elgin',
       'Alice', 'Bucyrus', 'Ammon', 'Cedar Hills', 'Ferriday', 'Fowler',
       'Groveland', 'Galesburg', 'Floresville', 'Gloucester City', 'Gramercy',
       'Glenwood', 'Camas', 'Faribault', 'Charleston', 'Cedarhurst',
       'Fountain Valley', 'Columbus', 'Dickson City', 'Alpine', 'Crookston',
       'Bay Minette', 'Garner', 'Grenoble', 'Catoosa', 'Farmers Branch',
       'Buckeye', 'El Campo', 'Erie', 'Cullman', 'Fargo', 'Bonita Springs',
       'Alamogordo', 'Elkton', 'Gas City', 'Fletcher', 'Davidson',
       'Florissant', 'Ellensburg', 'Center Point', 'Alamo Heights', 'Giffnock',
       'Florida City', 'Grayslake', 'Altus', 'El Monte', 'Campbellsville',
       'Camberwell', 'Glenn Heights', 'Flower Mound', 'Benton Harbor', 'Canby',
       'Coleraine', 'Fountain Inn', 'Gillespie', 'Elkin', 'Gaffney',
       'Bensenville', 'Carteret', 'Bryan'],

In [88]:
f=pd.read_csv('LRU_order_among_the_years.csv', header=1)


# delete the first line 
f=f.drop(f.index[0])
f.reset_index(drop=True, inplace=True)

# List to save every prediction errors and we can calculate their mean as overall error afterwards
error_list = []

best_list=[]
moyenne_list=[]
worst_list=[]

#choose the spare part we want, delete other colomns  and change the formula to do ''Prophet'' training
for PARA in PARAs:
    df=f[['LRU',PARA]]
    df['ds'] = df['LRU']
    df['y'] = df[PARA]
    df = df.drop(['LRU',PARA],axis=1)
    ## do the sum of commandes and get the new list  
    Sum_Aberaeron = df
    Sum_Aberaeron.reset_index(drop=True, inplace=True)
    Sum_Aberaeron['ds'] = pd.to_datetime(Sum_Aberaeron['ds'])
    Sum_Aberaeron.sort_values('ds',inplace=True)
    Sum_Aberaeron['cumsum']=Sum_Aberaeron['y'].cumsum()
    Sum_Aberaeron['y']=Sum_Aberaeron['cumsum']
    Sum_Aberaeron  =Sum_Aberaeron.drop(['cumsum'],axis=1)
    
    # split the training set and test set: training :'from 2009 to '20171231' , testing :'from 20180101 to '20180926' 
    train=Sum_Aberaeron[Sum_Aberaeron['ds']<'20180101']
    test=Sum_Aberaeron[Sum_Aberaeron['ds']>'20171231']
    
    #prophet traing
    m = Prophet(growth='linear', changepoints=None, n_changepoints=25, changepoint_range=0.5, yearly_seasonality='auto', weekly_seasonality='auto', daily_seasonality='auto', holidays=None, seasonality_mode='additive', seasonality_prior_scale=10.0, holidays_prior_scale=10.0, changepoint_prior_scale=0.05, mcmc_samples=0, interval_width=0.8, uncertainty_samples=1000)
    m.fit(train)
    forecast = m.predict(test)
    
 # calculate the grow rate in year 2018  for both the true data and prediction
    firstday = forecast[forecast['ds']=='20180101']
    lastday = forecast[forecast['ds']=='20180926']
    agg_2018_pre= (lastday['yhat'].values[0] - firstday['yhat'].values[0])
    #print(PARA+' predicted command number in 2018 : ' +str(agg_2018_pre))

    # print("Generalization error: %g"%(1.-rf.score(Xtest,ytest)))

    firstday_test = test[test['ds']=='20180101']
    lastday_test = test[test['ds']=='20180926']
    agg_2018 = (lastday_test['y'].values[0] - firstday_test['y'].values[0])
    #print(PARA+' true command number in 2018: ' + str(agg_2018))

    #calculate the prediction error
    error=np.abs(agg_2018-agg_2018_pre)/agg_2018 
    if error<0.1:
        best_list.append(PARA)
    else:
        if error<0.4:
            moyenne_list.append(PARA)
        else:
            worst_list.append(PARA)
    print(PARA+' PREDICT_error: ' + str(error ))
    error_list.append(error)

/Users/qiuqiuli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/qiuqiuli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Alliance PREDICT_error: 0.9872462401506036


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Ankeny PREDICT_error: 1.7773681401142092


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

Gaffney PREDICT_error: 0.04695379797160228


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Bensenville PREDICT_error: 0.5426667919021052


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Carteret PREDICT_error: 0.43488720809359954
Bryan PREDICT_error: 0.5455078109884921


In [77]:
error_list.sort()
error_list.pop(-1)
error_list.pop(-1)
error_list.pop(-1)
error_list

[0.0062275814704968015,
 0.01206597863444823,
 0.04695379797160228,
 0.04720470086174089,
 0.04725839905301657,
 0.05153874569727584,
 0.0602511359897347,
 0.06851981716203684,
 0.0789737344378409,
 0.08444966506451337,
 0.10773783688140424,
 0.1381872295986009,
 0.14309700595054053,
 0.1898727159754592,
 0.19052662319514974,
 0.2352836620478167,
 0.23643749079426984,
 0.2370611896717557,
 0.23765648988270627,
 0.24217486426895823,
 0.24735003347304196,
 0.29648991004500214,
 0.3061434667659555,
 0.31830602001090114,
 0.3262196961719849,
 0.36496871062847164,
 0.37139017173413863,
 0.38970763344416987,
 0.3981237914192197,
 0.4168289617725054,
 0.4182280999633181,
 0.43488720809359954,
 0.4608741397793262,
 0.4842863876451526,
 0.4897097895239968,
 0.538846673018393,
 0.5399606341643237,
 0.5401495176237325,
 0.5426667919021052,
 0.5455078109884921,
 0.5564555779614936,
 0.5704905639038165,
 0.5767449130836112,
 0.5822975684239093,
 0.6507197936459088,
 0.7120912079083861,
 0.723824016

In [78]:
print(np.mean(error_list))
print(np.std(error_list))

0.5863334188757254
0.48266399917582287


In [89]:
best_list

['Columbiana',
 'Cedar Hills',
 'Cedarhurst',
 'Dickson City',
 'Bay Minette',
 'El Campo',
 'Flower Mound',
 'Coleraine',
 'Gillespie',
 'Gaffney']

In [90]:
moyenne_list

['Greensboro',
 'Clifton',
 'Elgin',
 'Ferriday',
 'Glenwood',
 'Faribault',
 'Crookston',
 'Garner',
 'Catoosa',
 'Farmers Branch',
 'Fargo',
 'Bonita Springs',
 'Gas City',
 'Florissant',
 'Ellensburg',
 'Center Point',
 'Giffnock',
 'Glenn Heights',
 'Canby']

In [91]:
worst_list

['Alliance',
 'Ankeny',
 'Darlington',
 'Chelan',
 'Dayton',
 'Cleveland Heights',
 'Alice',
 'Bucyrus',
 'Ammon',
 'Fowler',
 'Groveland',
 'Galesburg',
 'Floresville',
 'Gloucester City',
 'Gramercy',
 'Camas',
 'Charleston',
 'Fountain Valley',
 'Columbus',
 'Alpine',
 'Grenoble',
 'Buckeye',
 'Erie',
 'Cullman',
 'Alamogordo',
 'Elkton',
 'Fletcher',
 'Davidson',
 'Alamo Heights',
 'Florida City',
 'Grayslake',
 'Altus',
 'El Monte',
 'Campbellsville',
 'Camberwell',
 'Benton Harbor',
 'Fountain Inn',
 'Elkin',
 'Bensenville',
 'Carteret',
 'Bryan']

# change_point_rate = 0.01 : 0.61 (delete the 3 dernieres) std 0.28
# change_point_rate = 0.2 : 0.57 (delete the 2 dernieres)
# change_point_rate = 0.5 : 0.59 (delete the 3 dernieres) std 0.48
# change_point_rate = 0.7 : 0.54 (delete the 3 dernieres) std 0.40
# change_point_rate = 0.8 : 0.62 (delete the 3 dernieres)